# Positional Encoding
------------------------------------------------
Transformer 및 후속 논문들에서는 word embedding과 더불어 positional embedding을 함께 사용한다. RNN과 같은 Auto-regressive 모델들에서 개별 토큰의 포지션은 sequence의 입력이 타임스텝 별로 하나의 토큰씩 feeding되며 포지션 인코딩이 모델에 내포된다.

<img src="https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2017/12/06022525/bptt.png" alt="Drawing" style="width: 500px;"/>

### Why positional encoding?
하지만 FFN계열의 Transformer와 같은 구조에서는 모든 인풋을 one time step에 받게 되어 추가적인 처리가 필요하다. 이 처리를 해주지 않았을 경우에 어떻게 될까? <a href="https://www.codecogs.com/eqnedit.php?latex=x_1" target="_blank"><br>
    <a href="https://www.codecogs.com/eqnedit.php?latex=input&space;X&space;=&space;[x_1,&space;x_2,&space;...x_l_e_n_(_t_o_k_e_n_)]" target="_blank"><img src="https://latex.codecogs.com/gif.latex?input&space;X&space;=&space;[x_1,&space;x_2,&space;...x_l_e_n_(_t_o_k_e_n_)]" title="input X = [x_1, x_2, ...x_l_e_n_(_t_o_k_e_n_)]" /></a><br>
위와 같은 순서로 이루어진 문장 혹은 정보가 있을 때, 모델은 해당 인풋들의 순서에 의한 정보를 잃은 채 학습을 하게되어, 더 복잡한 _problem set_을 해결해야 하는 것과 마찬가지다. 언어와 같이 단어의 위치적 정보가 문장의 의미에 중요한 차이점을 야기하는 모델에 있어 이를 해결하는 것은 중요하다.<br>
### Different approaches to positional encoding
Input vector의 포지션을 기억하는 방식에도 여러가지 접근 방법이 있다. ConvS2S에서와 같이 absolute position을 할당하는 방식, 'Attention is all you need!'의 "sine and cosine functions of different frequencies"와 같이 relative position을 주는 방식의 absolute/relative 범주로 나뉜다. 또한, 'Improving Language Understanding by Generative Pre-Training'에서의 learnable positional encoding 등 learnable/fixed 범주로 나뉜다. Transformer의 Sinusodial 방식은 가변적인 길이의 굉장히 긴 문장의 입력에도 효과적으로 적용할 수 있다는 장점이 있으며, 이를 사용해보기로 한다.<br>
### Sinusodial positional encoding
<a href="https://www.codecogs.com/eqnedit.php?latex=PE(pos,2i)=sin(pos/10000^2^i^/^d^m^_odel)" target="_blank"><img src="https://latex.codecogs.com/gif.latex?PE(pos,2i)=sin(pos/10000^2^i^/^d^m^_odel)" title="PE(pos,2i)=sin(pos/10000^2^i^/^d^m^_odel)" /></a><a href="https://www.codecogs.com/eqnedit.php?latex=PE(pos,2i)=cos(pos/10000^2^i^/^d^m^_odel)" target="_blank"><img src="https://latex.codecogs.com/gif.latex?PE(pos,2i)=cos(pos/10000^2^i^/^d^m^_odel)" title="PE(pos,2i)=cos(pos/10000^2^i^/^d^m^_odel)" /></a><br>
Sinusodial 포지셔널 인코딩.. 이게 어떻게 돌아가는지 알아보자. 여기서 i는 dimension의 인덱스, dmodel은 모델의 dimension, pos는 word vector에 적용될 각 포지션의 인덱스이다. 여기까지는 논문에 잘 나와있는 설명이다. 근데 PE의 argument는 2i, 2i+1인데, sin, cos의 argument는 다시 2i?? 이게 무슨 의미인지 좀 더 자세히 파헤쳐보자. dmodel 은 embedding dimension의 크기 T이다. N은 우리가 모델에서 가정하는 max sequence length라 하자. 그렇다면, PE는 (N × T) 의 output 매트릭스를 내놓는다. i는 dimension의 index이기 때문에 max(i) = dmodel 과 같다.  코드로 보면,<br>
<br>
    `[[pos / 10,000^(2×(t//2)/d_model) for t in range(d_model)] if pos != 0 else [0 for t in range(d_model)] for pos in range(max_length)]`<br>
<br>
여기서 각 position n은 size T 만큼의 position embedding을 가지면, t // 2 == 0 인 경우 sin값을, t // 2 == 1 인 경우 cos 값을 가진다. wavelength는 t = d_model 일 때  10000 × 2pie ~ t = 0 일 때 2pie 까지의 범위를 가진다. 이렇게 positional encoding matrix를 구했다면, 남은 것은 text embedding에서 구한 embedding에 더해주는 것이다.
_**(Attention is all you need!의 저자는 합연산이 아닌 concatenation 방식도 시도해봤는데, 최종적으로는 더해주는 방식을 채용)**_

# Word Embedding
-------------------------------------
### One-hot encoding
단어, 혹은 토큰들은 문자형이다. 각각의 토큰 스트링들은 모두 고유한 값으로 표현될 수 있다. 이러한 개념을 이용한 것이 토큰의 집합이 있을 때, 집합의 크기를 차원의 수로 설정해, 개별 원소들을 각 차원에 1, 0으로 매핑하는 one-hot encoding이다. 하지만, one-hot encoding 방식은 토큰 집합에 정비례해 증가하는 차원수의 sparse matrix를 가지기에 큰 토큰 집합에서 비효율적이다.
<img src="https://www.tensorflow.org/images/feature_columns/categorical_column_with_identity.jpg" alt="Drawing" style="width: 300px;"/>


### Continuous embedding
Continuous embedding은 [이곳](https://en.wikipedia.org/wiki/Continuous_embedding)에서 설명되어 있듯이 continuous function에 의해서 다른 vector space로의 변환을 한다. 토큰들을 구분하는데 있어 {0, 1} 이라는 discrete한 정보로 구분하는 것이 아닌 \[0, 1\] 범위의 continuous한 값으로 구분이 되기 때문에 토큰 집합의 사이즈 만큼의 차원 수가 더 이상 필요하지 않다. 텍스트 데이터를 임베딩하는 방식에는 word2vec의 CBOW, n-gram 등을 이용할 수도 있고, 임의의 어떤 다른 모델을 이용해도 무방하다. 'Improving Language Understanding by Generative Pre-Training'에서는 transformer의 decoder 파트 자체를 임베딩 모델로 사용한다.

# Positional encoding code explanation
----------------------------------------
마지막부터 설명하자면, 전체모델에 한 번에 들어갈 수 있는 최대 토큰의 수(max length)와 임베딩 차원(embedding_dim)의 크기를 가지는 매트릭스를 아웃풋으로 가진다<br>

In [28]:
import numpy as np

max_len, embedding_dim = 512, 256
dim = embedding_dim
pe = np.zeros((max_len, embedding_dim))

토큰의 포지션, 임베딩 차원 별로 포지셔널 점수 할당

In [29]:
position = [[pos / 10000**(2*(t//2)/embedding_dim) for t in range(embedding_dim)] if pos != 0 else [0 for t in range(embedding_dim)] for pos in range(max_len)]

sin, cos 함수 적용

In [39]:
for i in range(max_len):
            for j in range(embedding_dim):
                if j % 2 ==0:
                    pe[i, j] = np.sin(position[i][j])
                else:
                    pe[i, j] = np.cos(position[i][j])